#### Capacities

##### Data ingestion strategy:
<mark style="background: #88D5FF;">**REPLACE**</mark>

##### Related pipeline:

**Load_Capacities_E2E**

##### Source:

**Files** from FUAM_Lakehouse folder **bronze_file_location** variable

##### Target:

**1 Delta table** in FUAM_Lakehouse 
- **gold_table_name** variable value


In [ ]:
from datetime import datetime, timedelta
from pyspark.sql.functions import col, explode
from delta.tables import *
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true") # needed for automatic schema evolution in merge 

In [ ]:
## Parameters
display_data = False

In [ ]:
## Variables
bronze_file_location = f"Files/raw/capacities/"
silver_table_name = "FUAM_Staging_Lakehouse.capacities_silver"
gold_table_name = "capacities"
gold_table_name_with_prefix = f"Tables/{gold_table_name}"

In [ ]:
# Clean Silver table, if exists
if spark.catalog.tableExists(silver_table_name):
    del_query = "DELETE FROM " + silver_table_name
    spark.sql(del_query)

In [ ]:
# Get Bronze data
bronze_df = spark.read.option("multiline", "true").json(bronze_file_location)

In [ ]:
if display_data:
    display(bronze_df)

In [ ]:
# Explode json subset structure
exploded_df = bronze_df.select(explode("value").alias("d"))

# Extract json objects to tabular form
extracted_df = exploded_df.select(col("d.*"))

if display_data:
    display(extracted_df)

In [ ]:
# Code generated by Data Wrangler for PySpark DataFrame

def clean_data(extracted_df):
    # Drop duplicate rows in column: 'region'
    extracted_df = extracted_df.dropDuplicates(['region'])
    return extracted_df

extracted_df_clean = clean_data(extracted_df)
display(extracted_df_clean)

In [ ]:
# Rename columns if needed
silver_df = extracted_df.withColumnRenamed("Id", "CapacityId")

if display_data:
    display(silver_df)

In [ ]:
# Write prepared bronze_df to silver delta table
silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(silver_table_name)

In [ ]:
# Get Silver table data
query = """
SELECT *
FROM """ + silver_table_name


silver_df = spark.sql(query)

if display_data:
     display(silver_df)

In [ ]:

# This function maps and merges the silver data to gold dynamically
def write_silver_to_gold(silver_table_name, gold_table_name, ids):
    query = "SELECT *, current_timestamp() AS fuam_modified_at, False as fuam_deleted  FROM " + silver_table_name 
    silver_df = spark.sql(query)
    
    if spark._jsparkSession.catalog().tableExists('FUAM_Lakehouse', gold_table_name):
        # if exists -> MERGE to gold
        print("Gold table exists and will be merged.")
        gold_df = DeltaTable.forName(spark, gold_table_name)


        gold_columns = gold_df.toDF().columns
        silver_columns = silver_df.columns
        combined_columns = list(set(gold_columns) | set(silver_columns))
        id_cols = {}
        merge_id_stmt = ''
        for col in combined_columns:
            if col in ids:
                merge_id_stmt =  merge_id_stmt +  " t." + col + " = s." + col + " and"
                id_cols[col] = "s." + col

                
        # delete last and in merge id statement
        merge_id_stmt = merge_id_stmt[:-4]


        # Merge silver (s = source) to gold (t = target)
        try:
            merge = (gold_df.alias('t') \
            .merge(silver_df.alias('s'), merge_id_stmt )) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .whenNotMatchedBySourceUpdate( condition = "t.fuam_deleted == False or t.fuam_deleted IS NULL", set = {"fuam_deleted" : "True", "fuam_modified_at": "current_timestamp()"} )
            
            merge.execute()
        except:
        # In case the tables already exist, but the fuam column are not existent because of an old version do merge whenNotMatchedBySourceUpdate
            merge = (gold_df.alias('t') \
            .merge(silver_df.alias('s'), merge_id_stmt )) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
                        
            merge.execute()

    else:
        # else -> INSERT to gold
        print("Gold table will be created.")

        silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(gold_table_name)

 

In [ ]:
# Merge semantic model refreshes to gold table
write_silver_to_gold(silver_table_name, gold_table_name, ['CapacityId'])

In [ ]:
# Write history of bronze files
mssparkutils.fs.cp(bronze_file_location, bronze_file_location.replace("Files/raw/", "Files/history/") + datetime.now().strftime('%Y/%m/%d') + "/", True)

In [ ]:
# Parse capacities admins and users into table capacities_users
df_capacity_users = spark.sql("""
SELECT CapacityId, 'Admin' as role , EXPLODE(admins) as user FROM FUAM_Lakehouse.capacities
UNION ALL
SELECT CapacityId, 'Contributor' as role, EXPLODE(users) as user FROM FUAM_Lakehouse.capacities
""")

df_capacity_users.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("capacity_users") 